# Pandas 1

## Name
Dylan Skinner

## Class

## Date
25 April 2023

In [1]:
import numpy as np
import pandas as pd

# Problem 1

In [25]:
# Prob 1
def prob1(file='budget.csv'):
    """"
    Read in budget.csv as a DataFrame with the index as column 0 and perform each of these operations on the DataFrame in order. 
    
    1) Reindex the columns such that amount spent on groceries is the first column and all other columns maintain the same ordering.
    2) Sort the DataFrame in descending order based on how much money was spent on Groceries.
    3) Reset all values in the 'Rent' column to 800.0.
    4) Reset all values in the first 5 data points to 0.0
    
    Return the values of the updated DataFrame as a NumPy array.
    
    Parameters:
        file (str): name of datafile
        
    Return:
        values (ndarray): values of DataFrame
    """
    # Read in file.
    budget = pd.read_csv(file, index_col=0)

    # Reindex the columns such that amount spent on groceries is the first column and all other columns maintain the same ordering.
    budget = budget.reindex(columns=["Groceries", "Rent", "Utilities", "Dining Out", "Gas", "Out With Friends", "Netflix"])

    # Sort df in descending order absed on how much money was spent on Groceries.
    budget = budget.sort_values('Groceries', ascending=False)
    # print(budget.head())

    # Reset all values in 'Rent' column to 800.0
    budget["Rent"] = 800.0
    
    # Reset all values in the first 5 data points to 0.0
    budget.iloc[:5, :] = 0.0
    
    # Return as np array
    return budget.to_numpy()


# print(prob1())

# Problem 2

In [54]:
# Prob 2
def prob2(file='budget.csv'):
    """
    Read in file as DataFrame.
    Fill all NaN values with 0.0.
    Create two new columns, 'Living Expenses' and 'Other'. 
    Sum the columns 'Rent', 'Groceries', 'Gas' and 'Utilities' and set it as the value of 'Living Expenses'.
    Sum the columns 'Dining Out', 'Out With Friends' and 'Netflix' and set as the value of 'Other'.
    Identify which column, other than 'Living Expenses' correlates most with 'Living Expenses'
    and which column other than 'Other' correlates most with 'Other'.

    Return the names of each of those columns as a tuple.
    The first should be of the column corresponding to \li{'Living Expenses'} and the second to \li{'Other'}.
    
    Parameters:
        file (str): name of datafile
        
    Return:
        values (tuple): (name of column that most relates to Living Expenses, name of column that most relates to Other)
    """
    # Read in the file.
    budget = pd.read_csv(file, index_col=0)
    
    # Fill all NaN values with 0.0
    budget = budget.fillna(0.0)

    # Create two new columns: 'Living Expenses' and 'Other'.
    budget['Living Expenses'] = budget[["Rent", 'Groceries', 'Gas', 'Utilities']].sum(axis=1)
    budget['Other'] = budget[['Dining Out', 'Out With Friends', 'Netflix']].sum(axis=1)

    # Identify which column, other than living expenses, correlates most with living expenses. Same for other.
    # Create correlation matrix
    correlation_matrix = budget.corr()
    
    # Set the 'Living Expenses', 'Living Expenses' and 'Other', 'Other' parts to be small.
    correlation_matrix.loc['Living Expenses', 'Living Expenses'] = -5.0
    correlation_matrix.loc['Other', 'Other'] = -5.0
    
    # Get the ID for highest correlation between for living expenses and other, return it
    return (correlation_matrix['Living Expenses'].idxmax(), correlation_matrix['Other'].idxmax())

('Rent', 'Dining Out')


# Problem 3

In [99]:
def prob3(file='crime_data.csv'):
    """
    Read in crime data and use pandas to answer the following questions.
    
    Set the index as the column 'Year', and return the answers to each question as a tuple.
    
    1) Identify the three crimes that have a mean over 1,500,000. 
    Of these three crimes, which two are very correlated? 
    Which of these two crimes has a greater maximum value?
    Save the title of this column as a variable to return as the answer.
    
    2) Examine the data since 2000.
    Sort this data (in ascending order) according to number of murders.
    Find the years where Aggravated Assault is greater than 850,000.
    Save the indices (the years) of the masked and reordered DataFrame as a NumPy array to return as the answer.
    
    3) What year had the highest crime rate? 
    In this year, which crime was committed the most? 
    What percentage of the total crime that year was it? 
    Save this value as a float.
    
    
    Parameters:
        file (str): data
    
    Return:
        ans_1 (string): answer to Question 1
        ans_2 (ndarray): answer to Question 2
        ans_3 (float): answer to Question 3
    """
    # Read in data (with year as the index).
    crime_data = pd.read_csv(file, index_col='Year')
    # print(crime_data)

    # Question 1:
    # Find the top 3 crimes.
    crime_mean = crime_data.mean(axis=0).drop(['Population', 'Total'])
    crime_mask = crime_mean >= 1500000
    top_3_crimes = crime_mean[crime_mask].index.to_list()

    # Find correlation between crimes.
    crime_corr = crime_data[top_3_crimes].corr()
    crime_corr[crime_corr == 1.0] = 0.0

    max_val = 0.0
    for crime in top_3_crimes:
        if crime_corr[crime].max() > max_val:
            max_pair = [crime, crime_corr.loc[crime].idxmax()]
    
    # Find which of these two crimes has the greatest maximal value
    question1 = crime_mean[max_pair].idxmax()

    # Question 2:
    # Get years 2000 and later.
    ind = crime_data.index.to_list()  # Get the index where 2000 is.
    crime_2000s = crime_data.iloc[ind.index(2000):]

    # Sort the data (in ascending order) by the number of murders.
    agg_mask = crime_2000s['Aggravated Assault'] > 850000
    crime_2000s = crime_2000s[agg_mask].sort_values(by="Murder", ascending=True)
    question2 = crime_2000s.index.to_numpy()
    
    # Question 3
    crime_data['Crime Rate'] = crime_data['Total'] / crime_data['Population']
    crime_rate_year = crime_data['Crime Rate'].idxmax()
    max_crime = crime_data.iloc[ind.index(crime_rate_year)][2:].idxmax()  # Crime that happened the most in the highest
                                                                          # crime rate year.
    question3 = 100*(crime_data.iloc[ind.index(crime_rate_year)][max_crime] / crime_data.iloc[ind.index(crime_rate_year)]['Total'])
    
    # Return the answers to questions 1-3
    return (question1, question2, question3)


('Property', array([2000, 2001, 2002, 2003, 2005, 2007, 2006]), 89.97188308734142)


# Problem 4

In [107]:
def prob4(file='DJIA.csv'):
    """

    Read the data with a DatetimeIndex as the index.
    Drop rows any rows without numerical values, cast the "VALUE" column to floats, then return the updated DataFrame.

    Parameters:
        file (str): data file
    Returns:
        df (DataFrame): updated DataFrame of stock market data
    """
    # Read in data
    djia = pd.read_csv(file, index_col='DATE')

    # Drop rows without numerical values
    djia["VALUE"] = pd.to_numeric(djia['VALUE'], errors='coerce')
    djia.dropna(inplace=True)

    # Return df
    return djia

print(prob4())

               VALUE
DATE                
2006-09-27  11689.24
2006-09-28  11718.45
2006-09-29  11679.07
2006-10-02  11670.35
2006-10-03  11727.34
...              ...
2016-09-20  18129.96
2016-09-21  18293.70
2016-09-22  18392.46
2016-09-23  18261.45
2016-09-26  18094.83

[2517 rows x 1 columns]


# Problem 5

In [116]:
def prob5(file='paychecks.csv'):
    """
    Create data_range for index of paycheck data.

    Parameters:
        file (str): data file
    Returns:
        df (DataFrame): DataFrame of paycheck data
    """
    # Read in data
    paychecks = pd.read_csv(file, header=None)

    # Create data range
    dates = pd.date_range(start="03/14/2008", periods=93, freq="14D")
    
    # Set dates as index of dataframe.
    paychecks.set_index(dates, inplace=True)
    
    return paychecks

                  0
2008-03-14  1122.26
2008-03-28   921.03
2008-04-11   962.46
2008-04-25  1035.97
2008-05-09  1078.59
...             ...
2011-07-29  1095.53
2011-08-12  1018.39
2011-08-26  1027.08
2011-09-09  1005.90
2011-09-23   963.29

[93 rows x 1 columns]


# Problem 6

In [118]:
def prob6(file='DJIA.csv'):
    """
    Compute the following information about the DJIA dataset
    1. The single day with the largest gain
    2. The single day with the largest loss

    Parameters:
        file (str): data file
    Returns:
        max_day (<M8[ns]): DateTimeIndex of maximum change
        min_day (<M8[ns]): DateTimeIndex of minimum change
    """
    # Get cleaned data from problem 4
    djia = prob4(file)

    # Get shift values
    shift = djia - djia.shift(1)

    # Find single day with largest gain.
    max_day = shift.idxmax()

    # Fin single day with largest loss.
    min_day = shift.idxmin()

    return max_day, min_day

print(prob6())

(VALUE    2008-10-13
dtype: object, VALUE    2008-09-29
dtype: object)
